In [1]:
import sys
print("SYS",sys.version)
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)

# tf.set_random_seed(42)
import keras.backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Embedding
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense, Activation, Dropout, RepeatVector
from keras.utils import np_utils
# from tf.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle

SYS 3.10.2 (main, Mar 16 2022, 10:43:30) [GCC 4.8.5 20150623 (Red Hat 4.8.5-44)]


2022-12-11 00:18:40.633250: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 00:18:41.209670: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-11 00:19:01.944131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/deeplearning/Python-3.10.2/libcutensor/lib/11:/N/soft/rhel7/deeplearning/Python-3.10.2/libcusparse_lt/lib64:/N/soft/rhel7/deeplearning/Python-3.10.2/cuda/lib64:/N/soft/rhel7/deeplearning/Pyt

2.10.0


In [2]:
model_file = "LSTM_embeddinglayer_dropout50_epochs70_final.dat"
mf=open(model_file,'rb')
model_pred = pickle.load(mf)

2022-12-11 00:20:00.580980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/deeplearning/Python-3.10.2/libcutensor/lib/11:/N/soft/rhel7/deeplearning/Python-3.10.2/libcusparse_lt/lib64:/N/soft/rhel7/deeplearning/Python-3.10.2/cuda/lib64:/N/soft/rhel7/deeplearning/Python-3.10.2/lib:/N/soft/rhel7/deeplearning/Python-3.10.2/:/N/soft/rhel7/cuda/11.5/lib64:/N/soft/rhel7/openmpi/intel/4.0.1/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/ipp/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/mkl/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/tbb/lib/intel64/gcc4.

In [3]:
# This is the training data
training_file ="train.tsv"

corpus = ""
for line in open(training_file).readlines():

	# Each line in the training data is two columns
	# тыкгаткэта ивнин риӄукэтэ	ты>кгат>кэ>та ив>ни>н риӄукэ>тэ

	row = line.strip().split('\t')

	# The tokens are in the first column
	tokens = row[0]

	# Add the tokens to the list of sentences

	corpus+=tokens


print(set(corpus))
chars = sorted(list(set(corpus)))
print(chars)
char_index = dict((c,i) for i,c in enumerate(chars))
index_char = dict((i,c) for i,c in enumerate(chars))
char_index["<UNK>"]=len(chars)
index_char[len(chars)]="<UNK>"
print(char_index)
print(index_char)

SEQUENCE_LENGTH = 10

{'ж', 'м', 'а', 'ӈ', 'з', 'ч', 'х', 'о', 'щ', 'в', 'и', 'ӄ', 'к', 'б', 'ы', 'ё', 'ф', 'у', 'д', 'с', 'ц', 'қ', 'э', 'ш', 'ʼ', '’', 'ъ', 'г', 'я', 'ю', 'ԓ', 'п', 'т', 'й', 'н', 'е', ' ', 'ң', 'ь', 'р'}
[' ', 'ʼ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'қ', 'ң', 'ӄ', 'ӈ', 'ԓ', '’']
{' ': 0, 'ʼ': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'м': 13, 'н': 14, 'о': 15, 'п': 16, 'р': 17, 'с': 18, 'т': 19, 'у': 20, 'ф': 21, 'х': 22, 'ц': 23, 'ч': 24, 'ш': 25, 'щ': 26, 'ъ': 27, 'ы': 28, 'ь': 29, 'э': 30, 'ю': 31, 'я': 32, 'ё': 33, 'қ': 34, 'ң': 35, 'ӄ': 36, 'ӈ': 37, 'ԓ': 38, '’': 39, '<UNK>': 40}
{0: ' ', 1: 'ʼ', 2: 'а', 3: 'б', 4: 'в', 5: 'г', 6: 'д', 7: 'е', 8: 'ж', 9: 'з', 10: 'и', 11: 'й', 12: 'к', 13: 'м', 14: 'н', 15: 'о', 16: 'п', 17: 'р', 18: 'с', 19: 'т', 20: 'у', 21: 'ф', 22: 'х', 23: 'ц', 24: 'ч', 25: 'ш', 26: 'щ', 

In [4]:
def predict_next_n_chars(pattern, n):
    output=""
    for i in range(n):
      x = np.reshape(pattern, (1, len(pattern), 1))
      prediction = model_pred.predict(x, verbose=0)
      output+=index_char[np.argmax(prediction)]
      seq_in = [index_char[value] for value in pattern]
      pattern.append(np.argmax(prediction))
      pattern = pattern[1:len(pattern)]
      if index_char[np.argmax(prediction)]==" ":
            break
            
    return output

In [5]:
def prepare_input(pattern):
    if len(pattern)>SEQUENCE_LENGTH:
      pattern=pattern[-10:]
      input_str =[char_index[value] for value in pattern]
    else:
      input_str =[char_index[value] for value in pattern]
      for j in range(len(input_str),SEQUENCE_LENGTH):
        input_str.append(0)
        
    return input_str

In [6]:
def predictive_text2(test_string):
    hits=0
    tokens=0
    clicks=0
    words=test_string.split(" ")
   # print("words",words)
    output = []
    tokens=len(words)
    pattern =''
    for word in words:
        if words[0]!=word:
            pattern+=" "
        for i in range(len(word)):
            three_preds=[]
            pattern+=word[i]
            input_str=prepare_input(pattern)
            #print("Input",input_str)
            #print("Pattern",pattern)
            x = np.reshape(input_str, (1, len(input_str), 1))
            #print("x",x.shape)
            pred=model_pred.predict(x)
            #print(pred)
            first=index_char[np.argmax(pred)]
            second = index_char[np.argsort(np.max(pred,axis=0))[-2]]
            third = index_char[np.argsort(np.max(pred,axis=0))[-3]]
#             print(first)
#             print(second)
#             print(third)
            first_word = predict_next_n_chars(prepare_input(pattern+first),20)
            second_word = predict_next_n_chars(prepare_input(pattern+second),20)
            third_word = predict_next_n_chars(prepare_input(pattern+third),20)
            first_pred = first+first_word
            second_pred = second+second_word
            third_pred = third+third_word
            actual=word[i+1:]
#             print("first",first_pred)
#             print("second",second_pred)
#             print("third",third_pred)
#             print("to be predicted",actual)
            if (actual).strip()== first_pred.strip() or (actual).strip()==second_pred.strip() or (actual).strip()==third_pred.split():
                output.append(actual.strip())
                output.append(" ")
                hits+=1
                clicks+=1
                break
            elif i<len(word)-1:
                output.append(word[i+1])
                clicks+=1
            else:
                output.append(" ")
                clicks+=1

    return [clicks,tokens,hits,[test_string,output]]


In [7]:
validate=open('dev.tsv','r').readlines()
validation_data=[]
for line in validate:
    lines=line.split('\t')
    text=lines[0]
    validation_data.append(text)

In [8]:
final_result=[]
total_hits=0
total_clicks=0
total_tokens=0
output_file2 = open("output_seq10_epochs70_dev_200.txt","a")
for line in validation_data[:200]:
    clicks,tokens,hits,outputs=predictive_text2(line)
    line,output=outputs
    output_file2.write(line+"   "+str(output)+"\n")
    #print(output)
    #print("hits",hits)
    final_result.append(outputs)
    total_hits+=hits
    total_clicks+=clicks
    total_tokens+=tokens

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 28ms/step


...

1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 28ms/step


In [18]:
print(total_hits)
print(total_clicks)
print(total_tokens)
print(total_clicks/total_tokens)

500
4515
786
5.744274809160306
